In [4]:
# -*- coding: utf-8 -*-
'''
==============================================================
Rozwiązywanie problemu opóźnienia całkowitego ważonego w systemie jednego stanowiska. 
Funkcja celu polega na minimalizacji całkowitego ważonego opóźnienia.
==============================================================
'''
# importowanie wymaganych modułów
import pandas as pd
import numpy as np
import time
import copy


''' ================= ustawienia inicjalizacyjne ======================'''
num_job = 20  # liczba zadań

p = [10, 10, 13, 4, 9, 4, 8, 15, 7, 1, 9, 3, 15, 9, 11, 6, 5, 14, 18, 3]
d = [50, 38, 49, 12, 20, 105, 73, 45, 6, 64, 15, 6, 92, 43, 78, 21, 15, 50, 150, 99]
w = [10, 5, 1, 5, 10, 1, 5, 10, 5, 1, 5, 10, 10, 5, 1, 10, 5, 5, 1, 5]
# wprowadzenie danych
population_size = int(input('Proszę podać rozmiar populacji: ') or 30)  # domyślna wartość to 30
crossover_rate = float(input('Proszę podać współczynnik krzyżowania: ') or 0.8)  # domyślna wartość to 0.8
mutation_rate = float(input('Proszę podać współczynnik mutacji: ') or 0.1)  # domyślna wartość to 0.1
mutation_selection_rate = float(input('Proszę podać współczynnik wyboru do mutacji: ') or 0.5)
num_mutation_jobs = round(num_job * mutation_selection_rate)
num_iteration = int(input('Proszę podać liczbę iteracji: ') or 2000)  # domyślna wartość to 2000

start_time = time.time()

'''==================== główny kod ==============================='''
'''----- generowanie początkowej populacji -----'''
Tbest = 999999999999999
best_list, best_obj = [], []
population_list = []
for i in range(population_size):
    random_num = list(np.random.permutation(num_job))  # generowanie losowej permutacji od 0 do num_job-1
    population_list.append(random_num)  # dodanie do listy populacji
        
for n in range(num_iteration):
    Tbest_now = 99999999999           
    '''-------- krzyżowanie --------'''
    parent_list = copy.deepcopy(population_list)
    offspring_list = copy.deepcopy(population_list)
    S = list(np.random.permutation(population_size))  # generowanie losowej sekwencji do wyboru chromosomów rodziców do krzyżowania
    
    for m in range(int(population_size / 2)):
        crossover_prob = np.random.rand()
        if crossover_rate >= crossover_prob:
            parent_1 = population_list[S[2 * m]][:]
            parent_2 = population_list[S[2 * m + 1]][:]
            child_1 = ['na' for i in range(num_job)]
            child_2 = ['na' for i in range(num_job)]
            fix_num = round(num_job / 2)
            g_fix = list(np.random.choice(num_job, fix_num, replace=False))
            
            for g in range(fix_num):
                child_1[g_fix[g]] = parent_2[g_fix[g]]
                child_2[g_fix[g]] = parent_1[g_fix[g]]
            c1 = [parent_1[i] for i in range(num_job) if parent_1[i] not in child_1]
            c2 = [parent_2[i] for i in range(num_job) if parent_2[i] not in child_2]
            
            for i in range(num_job - fix_num):
                child_1[child_1.index('na')] = c1[i]
                child_2[child_2.index('na')] = c2[i]
            offspring_list[S[2 * m]] = child_1[:]
            offspring_list[S[2 * m + 1]] = child_2[:]
        
    '''--------mutacja--------'''   
    for m in range(len(offspring_list)):
        mutation_prob = np.random.rand()
        if mutation_rate >= mutation_prob:
            m_chg = list(np.random.choice(num_job, num_mutation_jobs, replace=False))  # wybór pozycji do mutacji
            t_value_last = offspring_list[m][m_chg[0]]  # zapisanie wartości na pierwszej pozycji do mutacji
            for i in range(num_mutation_jobs - 1):
                offspring_list[m][m_chg[i]] = offspring_list[m][m_chg[i + 1]]  # przesunięcie wartości
            
            offspring_list[m][m_chg[num_mutation_jobs - 1]] = t_value_last  # przeniesienie wartości pierwszej pozycji do ostatniej pozycji mutacji
    
    '''--------wartość dopasowania (obliczanie opóźnienia)-------------'''
    total_chromosome = copy.deepcopy(parent_list) + copy.deepcopy(offspring_list)  # kombinacja chromosomów rodziców i potomków
    chrom_fitness, chrom_fit = [], []
    total_fitness = 0
    for i in range(population_size * 2):
        ptime = 0
        tardiness = 0
        for j in range(num_job):
            ptime = ptime + p[total_chromosome[i][j]]
            tardiness = tardiness + w[total_chromosome[i][j]] * max(ptime - d[total_chromosome[i][j]], 0)
        chrom_fitness.append(1 / tardiness)
        chrom_fit.append(tardiness)
        total_fitness = total_fitness + chrom_fitness[i]
    
    '''----------selekcja----------'''
    pk, qk = [], []
    
    for i in range(population_size * 2):
        pk.append(chrom_fitness[i] / total_fitness)
    for i in range(population_size * 2):
        cumulative = 0
        for j in range(0, i + 1):
            cumulative = cumulative + pk[j]
        qk.append(cumulative)
    
    selection_rand = [np.random.rand() for i in range(population_size)]
    
    for i in range(population_size):
        if selection_rand[i] <= qk[0]:
            population_list[i] = copy.deepcopy(total_chromosome[0])
        else:
            for j in range(0, population_size * 2 - 1):
                if selection_rand[i] > qk[j] and selection_rand[i] <= qk[j + 1]:
                    population_list[i] = copy.deepcopy(total_chromosome[j + 1])
                    break
    '''----------porównanie----------'''
    for i in range(population_size * 2):
        if chrom_fit[i] < Tbest_now:
            Tbest_now = chrom_fit[i]
            sequence_now = copy.deepcopy(total_chromosome[i])
    
    if Tbest_now <= Tbest:
        Tbest = Tbest_now
        sequence_best = copy.deepcopy(sequence_now)
    
    job_sequence_ptime = 0
    num_tardy = 0
    for k in range(num_job):
        job_sequence_ptime = job_sequence_ptime + p[sequence_best[k]]
        if job_sequence_ptime > d[sequence_best[k]]:
            num_tardy = num_tardy + 1
'''----------wyniki----------'''
print("optymalna sekwencja", sequence_best)
print("optymalna wartość: %f" % Tbest)
print("średnie opóźnienie: %f" % (Tbest / num_job))
print("liczba opóźnionych: %d" % num_tardy)
print('czas wykonania: %s' % (time.time() - start_time))


optymalna sekwencja [np.int32(11), np.int32(15), np.int32(4), np.int32(16), np.int32(3), np.int32(9), np.int32(7), np.int32(0), np.int32(8), np.int32(13), np.int32(1), np.int32(12), np.int32(6), np.int32(19), np.int32(10), np.int32(17), np.int32(5), np.int32(2), np.int32(18), np.int32(14)]
optymalna wartość: 2062.000000
średnie opóźnienie: 103.100000
liczba opóźnionych: 15
czas wykonania: 26.50299644470215


In [5]:
'''--------rysowanie wykresu Gantta-------'''
import pandas as pd
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.offline as offline
import datetime

# Klucze zadań
j_keys = [j for j in range(num_job)]
j_count = {key: 0 for key in j_keys}
m_count = 0
j_record = {}

# Obliczanie czasu rozpoczęcia i zakończenia dla każdego zadania
for i in sequence_best:
    gen_t = int(p[i])
    j_count[i] = j_count[i] + gen_t
    m_count = m_count + gen_t

    if m_count < j_count[i]:
        m_count = j_count[i]
    elif m_count > j_count[i]:
        j_count[i] = m_count
    start_time = str(datetime.timedelta(seconds=j_count[i] - p[i]))  # konwersja sekund na godziny, minuty i sekundy

    end_time = str(datetime.timedelta(seconds=j_count[i]))
    j_record[i] = [start_time, end_time]

# Tworzenie danych dla wykresu Gantta
df = []
for j in j_keys:
    df.append(dict(Task='Machine', Start='2018-07-14 %s' % (str(j_record[j][0])), Finish='2018-07-14 %s' % (str(j_record[j][1])), Resource='Job %s' % (j + 1)))

# Kolory dla zadań (opcjonalne)
# colors={}
# for i in j_keys:
#     colors['Job %s' % (i + 1)] = 'rgb(%s,%s,%s)' % (255 / (i + 1) + 0 * i, 5 + 12 * i, 50 + 10 * i)

# Tworzenie wykresu Gantta
fig = ff.create_gantt(df, colors=['#008B00', '#FF8C00', '#E3CF57', '#0000CD', '#7AC5CD', '#ED9121', '#76EE00', '#6495ED', '#008B8B', '#A9A9A9', '#A2CD5A', '#9A32CD', '#8FBC8F', '#EEC900', '#EEE685', '#CDC1C5', '#9AC0CD', '#EEA2AD', '#00FA9A', '#CDB38B'], index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True)

# Wyświetlanie wykresu w trybie offline
offline.iplot(fig, filename='GA_flow_shop_scheduling_tardyjob')


In [10]:
# -*- coding: utf-8 -*-
'''==========Rozwiązanie problemu harmonogramowania przepływu za pomocą algorytmu genetycznego w Pythonie======='''
# importowanie wymaganych modułów
#import os
import pandas as pd
import numpy as np
import time
import copy

''' ================= ustawienia inicjalizacji ======================'''
#pt_tmp = pd.read_excel("20x5_flowshop.xlsx", sheet_name="S1", index_col=[0])
pt_tmp = pd.read_excel("9x5_flowshop.xlsx", sheet_name="S1", index_col=[0])
pt = pt_tmp.values.tolist()
num_m = 5
num_job = len(pt)
# raw_input jest używane w Pythonie 2
rozmiar_populacji = int(input('Podaj rozmiar populacji: ') or 30) # domyślna wartość to 30
crossover_rate = float(input('Podaj współczynnik krzyżowania: ') or 0.8) # domyślna wartość to 0.8
mutation_rate = float(input('Podaj współczynnik mutacji: ') or 0.2) # domyślna wartość to 0.2
mutation_selection_rate = float(input('Podaj współczynnik selekcji mutacji: ') or 0.2)
num_mutation_jobs = round(num_job * mutation_selection_rate)
num_iteration = int(input('Podaj liczbę iteracji: ') or 2000) # domyślna wartość to 2000

start_time = time.time()

'''==================== główny kod ==============================='''
'''----- generowanie początkowej populacji -----'''
Tbest = 999999999999999
best_list, best_obj = [], []
population_list = []
for i in range(rozmiar_populacji):
    random_num = list(np.random.permutation(num_job)) # generowanie losowej permutacji od 0 do num_job-1
    population_list.append(random_num) # dodawanie do listy populacji

for n in range(num_iteration):
    Tbest_now = 99999999999
    '''-------- krzyżowanie --------'''
    parent_list = copy.deepcopy(population_list)
    offspring_list = copy.deepcopy(population_list)
    S = list(np.random.permutation(rozmiar_populacji)) # generowanie losowej sekwencji do wyboru rodzica do krzyżowania

    for m in range(int(rozmiar_populacji / 2)):
        crossover_prob = np.random.rand()
        if crossover_rate >= crossover_prob:
            parent_1 = population_list[S[2 * m]][:]
            parent_2 = population_list[S[2 * m + 1]][:]
            child_1 = ['na' for i in range(num_job)]
            child_2 = ['na' for i in range(num_job)]
            fix_num = round(num_job / 2)
            g_fix = list(np.random.choice(num_job, fix_num, replace=False))

            for g in range(fix_num):
                child_1[g_fix[g]] = parent_2[g_fix[g]]
                child_2[g_fix[g]] = parent_1[g_fix[g]]
            c1 = [parent_1[i] for i in range(num_job) if parent_1[i] not in child_1]
            c2 = [parent_2[i] for i in range(num_job) if parent_2[i] not in child_2]

            for i in range(num_job - fix_num):
                child_1[child_1.index('na')] = c1[i]
                child_2[child_2.index('na')] = c2[i]
            offspring_list[S[2 * m]] = child_1[:]
            offspring_list[S[2 * m + 1]] = child_2[:]

    '''-------- mutacja --------'''
    for m in range(len(offspring_list)):
        mutation_prob = np.random.rand()
        if mutation_rate >= mutation_prob:
            m_chg = list(np.random.choice(num_job, num_mutation_jobs, replace=False)) # wybór pozycji do mutacji
            t_value_last = offspring_list[m][m_chg[0]] # zapisanie wartości z pierwszej pozycji mutacji
            for i in range(num_mutation_jobs - 1):
                offspring_list[m][m_chg[i]] = offspring_list[m][m_chg[i + 1]] # przesunięcie

            offspring_list[m][m_chg[num_mutation_jobs - 1]] = t_value_last # przeniesienie wartości z pierwszej pozycji mutacji na ostatnią pozycję mutacji

    '''-------- wartość dopasowania (obliczanie czasu bezczynności) -------------'''

    total_chromosome = copy.deepcopy(parent_list) + copy.deepcopy(offspring_list) # połączenie chromosomów rodziców i potomków
    chrom_fitness, chrom_fit = [], []
    total_fitness = 0
    s, d, D = {}, {}, {}
    v = [0 for c in range(rozmiar_populacji * 2)]

    for c in range(rozmiar_populacji * 2):
        for i in range(num_m):
            s[c, i] = pt[total_chromosome[c][0]][i]
            d[c, i] = v[c]
            D[c, i, total_chromosome[c][0]] = v[c]
            v[c] = v[c] + pt[total_chromosome[c][0]][i]

        for j in range(num_job):
            D[c, 0, j] = 0

        for j in range(1, num_job):
            for i in range(0, num_m - 1):
                s[c, i] = s[c, i] + pt[total_chromosome[c][j]][i]
                D[c, i + 1, j] = max(0, s[c, i] + d[c, i] - s[c, i + 1] - d[c, i + 1])
                d[c, i + 1] = d[c, i + 1] + D[c, i + 1, j]

            s[c, num_m - 1] = s[c, num_m - 1] + pt[total_chromosome[c][j]][i + 1]

        v[c] = 0
        for i in range(num_m):
            v[c] = v[c] + d[c, i]

        chrom_fitness.append(1 / v[c])
        chrom_fit.append(v[c])
        total_fitness = total_fitness + chrom_fitness[c]

    '''---------- selekcja ----------'''
    pk, qk = [], []

    for i in range(rozmiar_populacji * 2):
        pk.append(chrom_fitness[i] / total_fitness)
    for i in range(rozmiar_populacji * 2):
        cumulative = 0
        for j in range(0, i + 1):
            cumulative = cumulative + pk[j]
        qk.append(cumulative)

    selection_rand = [np.random.rand() for i in range(rozmiar_populacji)]

    for i in range(rozmiar_populacji):
        if selection_rand[i] <= qk[0]:
            population_list[i] = copy.deepcopy(total_chromosome[0])
        else:
            for j in range(0, rozmiar_populacji * 2 - 1):
                if selection_rand[i] > qk[j] and selection_rand[i] <= qk[j + 1]:
                    population_list[i] = copy.deepcopy(total_chromosome[j + 1])
                    break

    '''---------- porównanie ----------'''
    for i in range(rozmiar_populacji * 2):
        if chrom_fit[i] < Tbest_now:
            Tbest_now = chrom_fit[i]
            sequence_now = copy.deepcopy(total_chromosome[i])

    if Tbest_now <= Tbest:
        Tbest = Tbest_now
        sequence_best = copy.deepcopy(sequence_now)

'''---------- wynik ----------'''
print("optymalna sekwencja", sequence_best)
print("optymalna wartość:%f" % Tbest)
print('czas trwania:%s' % (time.time() - start_time))


optymalna sekwencja [np.int32(1), np.int32(6), np.int32(0), np.int32(8), np.int32(4), np.int32(2), np.int32(3), np.int32(5), np.int32(7)]
optymalna wartość:160.000000
czas trwania:507.62012219429016


In [11]:
'''--------wykres Gantta-------'''
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
import datetime

# Definiowanie kluczy dla maszyn i zadań
m_keys = ['Maszyna %s' % (j + 1) for j in range(num_m)]
j_keys = [j for j in range(num_job)]
j_count = {key: 0 for key in j_keys}
m_count = {key: 0 for key in m_keys}
j_record = {}

# Obliczanie czasów rozpoczęcia i zakończenia dla każdego zadania
for i in sequence_best:
    for j in range(num_m):
        gen_t = int(pt[i][j])
        j_count[i] += gen_t
        m_count[m_keys[j]] += gen_t
        
        if m_count[m_keys[j]] < j_count[i]:
            m_count[m_keys[j]] = j_count[i]
        elif m_count[m_keys[j]] > j_count[i]:
            j_count[i] = m_count[m_keys[j]]
        
        start_time = str(datetime.timedelta(seconds=j_count[i] - pt[i][j]))  # Konwersja sekund na godziny, minuty i sekundy
        end_time = str(datetime.timedelta(seconds=j_count[i]))
        j_record[(i, m_keys[j])] = [start_time, end_time]

# Przygotowanie danych do wykresu
df = []
for m in m_keys:
    for j in j_keys:
        df.append(dict(Task=m, Start='2018-07-14 %s' % (str(j_record[(j, m)][0])), Finish='2018-07-14 %s' % (str(j_record[(j, m)][1])), Resource='%s' % (j)))

# Przygotowanie kolorów dla zadań
colors = {}
for i in j_keys:
    colors[str(i)] = 'rgb(%s,%s,%s)' % (255 / (i + 1) + 0 * i, 25 + 25 * i, 50 + 20 * i)

# Tworzenie wykresu Gantta
fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True)

# Wyświetlanie wykresu
fig.show()
